# Topic Modelling using BERTopic & cuBERTopic

Sample notebook to show cuBERTopic, a topic modelling technique that is built on top of the NVIDIA RAPIDS ecoysystem, utilizing libraries such as `cudf` and `cuml` to GPU-accelarate end-to-end workflow for extracting topic from a set of documents. We run the same operations using `BERTopic` to compare their behaviour. 

## Quick Start
In both the cases, we start by extracting topics from the well-known 20 newsgroups dataset from `sklearn` which is comprised of english documents

### Installing relevant packages
Here we need to install relevant dependencies for `BERTopic` as well as we compare performance between it and `cuBERTopic`. 

More detailed instructions are in the README.

In [1]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from transformers import AutoTokenizer, AutoModel
import torch
from cuBERTopic import gpu_BERTopic
import rmm
rmm.reinitialize(pool_allocator=True,initial_pool_size=5e+9)

docs = fetch_20newsgroups(subset='all')['data']

### Running `BERTopic`
`BERTopic` provides us the functionality of providing custom embeddings, so we create sentence embeddings using `AutoTokenizer` followed by `AutoModel` from `transformers` and pass it to `fit_transform` method inside `BERTopic` class, which fits the models on a collection of documents, generate topics, and return the docs with topics.

In [2]:
len(docs)

18846

In [3]:
%%time
# Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2"
)

# Tokenize sentences
encoded_input = tokenizer(
    docs,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask
        .unsqueeze(-1)
        .expand(token_embeddings.size())
        .float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(
    model_output,
    encoded_input["attention_mask"]
)
sentence_embeddings = sentence_embeddings.to('cpu').numpy()
topic_model = BERTopic()
topics_cpu, probs_cpu = topic_model.fit_transform(docs, sentence_embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

`get_topic_info` returns information about each topic including its id, frequency, and name 

In [4]:
%%time
topic_model.get_topic_info()

CPU times: user 0 ns, sys: 4.88 ms, total: 4.88 ms
Wall time: 4.15 ms


,Topic,Count,Name
0,-1,6236,-1_drive_information_anyone_distribution
1,0,498,0_monitor_card_video_drivers
2,1,251,1_car_cars_ford_engine
3,2,162,2_bike_bikes_honda_motorcycle
4,3,157,3_clipper_encryption_chip_key
...,...,...,...
364,381,11,381_clutch_sabo_nonclutch_samuel
363,372,11,372_okcforum_okcforumosrheedu_conner_bilokcfor...
383,382,10,382_eugenics_genes_memes_nazi
384,383,10,383_sin_heaven_sanctifying_doctrine


`get_topic` returns topics with top n words and their c-TF-IDF score

-1 refers to all outliers and should typically be ignored. Next, let's take a look at the most frequent topic that was generated, topic 0:

In [5]:
%%time
topic_model.get_topic(0)

CPU times: user 8 µs, sys: 5 µs, total: 13 µs
Wall time: 24.6 µs


[('monitor', 0.013231089739198978),
 ('card', 0.012511475814230476),
 ('video', 0.011350966056695404),
 ('drivers', 0.008468496228775997),
 ('vga', 0.007764846225710292),
 ('monitors', 0.006876280900935278),
 ('diamond', 0.006578098617890783),
 ('ati', 0.006485434989599792),
 ('screen', 0.00528880788779554),
 ('driver', 0.004991756577900355)]

### Running `cuBERTopic`
`cuBERTopic` provides with a similar API for passing in `docs` as a set of strings to model on. Here, instead of using `AutoTokenizer` from `transformers`, we use `SubwordTokenizer` from `cuDF` in combination with `AutoModel` from `transformers`. 

Due to the stochastisch nature of UMAP, the results might differ and the quality can degrade.

In [6]:
%%time
gpu_topic = gpu_BERTopic()
topics_gpu, probs_gpu = gpu_topic.fit_transform(docs)

Label prop iterations: 20
Label prop iterations: 8
Label prop iterations: 5
Label prop iterations: 5
Label prop iterations: 2
Label prop iterations: 2
Iterations: 6
1931,232,383,19,274,1317
Label prop iterations: 3
Label prop iterations: 3
Iterations: 2
1150,74,159,9,89,206
CPU times: user 25.6 s, sys: 5.19 s, total: 30.8 s
Wall time: 23.1 s


In [7]:
%%time
gpu_topic.get_topic_info()

CPU times: user 13.1 ms, sys: 2.3 ms, total: 15.4 ms
Wall time: 13.8 ms


,Topic,Count,Name
0,-1,6324,-1_information_drive_email_god
77,0,432,0_monitor_card_video_drivers
201,1,266,1_car_cars_engine_mustang
226,2,234,2_hockey_team_blues_game
33,3,175,3_printer_deskjet_print_hp
...,...,...,...
334,385,10,385_mormon_temple_ceremonies_temples
350,386,10,386_smiley_object_kuiper_karla
375,387,10,387_mormon_mormons_cjf_book
381,388,10,388_moon_luna_lunar_coffman


In [8]:
%%time
gpu_topic.get_topic(0)

CPU times: user 119 ms, sys: 15.2 ms, total: 134 ms
Wall time: 133 ms


[('monitor', array(0.01598659)),
 ('card', array(0.01449095)),
 ('video', array(0.01236304)),
 ('drivers', array(0.01076605)),
 ('vga', array(0.00952706)),
 ('monitors', array(0.00868088)),
 ('ati', array(0.0079947)),
 ('diamond', array(0.00798902)),
 ('screen', array(0.0063153)),
 ('vesa', array(0.00621126))]